# VENTAS GNC 2020 - 2025

### Preparamos el entorno y cargamos el archivo

In [ ]:
#Instalar las librerías necesarias
!pip install pandas numpy matplotlib


### Cargar archivo, rellenar y editar campos nuevos.

#### Usaremos los archivos csv ***campos_prod y ventas*** de nuestra carpeta

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np

# ============================
#  CARGA DE ARCHIVOS
# ============================

ruta_csv = r"C:\Users\DeVan\Documents\Jupyterlab\ventas\ventas.csv"
ruta_prod = r"C:\Users\DeVan\Documents\Jupyterlab\ventas\campos_prod.csv"

df = pd.read_csv(ruta_csv, sep=";", encoding="latin-1")
df_prod = pd.read_csv(ruta_prod, sep=";", encoding="latin-1")

columnas_originales = df.columns.tolist()


# ============================
#  MERGE PRINCIPAL
# ============================

df = df.merge(
    df_prod[['id_oceges', 'id', 'Unidades']],
    left_on='product_id',
    right_on='id_oceges',
    how='left'
)

df = df.rename(columns={
    'Unidades': 'unidad',
    'id': 'product_id_estandar'
})


# ============================
#  FECHAS
# ============================

for campo in ['fecha_venta', 'fecha_creacion']:
    if campo in df.columns:
        df[campo] = pd.to_datetime(df[campo], errors='coerce', dayfirst=True)


# ============================
#  NÚMEROS
# ============================

def limpiar_num(valor):
    return (
        valor.astype(str)
             .str.replace(",", ".", regex=False)
             .str.replace("€", "", regex=False)
             .str.strip()
    )

campos_float = [
    'importe_pagado','base_ud_sin_dto','importe_descuento','base_ud','base_total',
    'importe_iva','importe_recargo_equivalencia','tasa_conversion_EUR','subtotal',
    'tipo_iva','unidades','unidad'
]

for campo in campos_float:
    if campo in df.columns:
        df[campo] = limpiar_num(df[campo])
        df[campo] = pd.to_numeric(df[campo], errors='coerce')


for campo in ['product_id', 'product_id_estandar']:
    if campo in df.columns:
        df[campo] = (
            df[campo].astype(str)
                     .str.replace(".0", "", regex=False)
                     .str.strip()
        )
        df[campo] = pd.to_numeric(df[campo], errors="coerce").astype("Int64")


# ============================
#  CÁLCULOS
# ============================

df['unidades_totales'] = df['unidad'] * df['unidades']

df['euros_totales'] = np.where(
    df['moneda'] == 'EUR',
    df['subtotal'],
    df['subtotal'] * df['tasa_conversion_EUR']
).round(2)


# ============================
#  AÑADIR sku_estandar
# ============================

df = df.merge(
    df_prod[['id_oceges', 'code']],
    left_on='product_id_estandar',
    right_on='id_oceges',
    how='left'
)

df['sku_estandar'] = df['code']

# eliminar columnas duplicadas o basura
cols_eliminar = [c for c in df.columns if c.startswith('id_oceges') or c in ['code']]
df = df.drop(columns=cols_eliminar)


# ============================
#  ORDEN FINAL
# ============================

cols = list(df.columns)

# --- 1) euros_totales después de moneda ---

cols.remove('euros_totales')
pos = cols.index('moneda') + 1
cols.insert(pos, 'euros_totales')

# --- 2) insertar bloque después de codigo ---

bloque = ['product_id_estandar', 'sku_estandar', 'unidad', 'unidades_totales']

for b in bloque:
    if b in cols:
        cols.remove(b)

pos_codigo = cols.index('codigo') + 1

for i, b in enumerate(bloque):
    cols.insert(pos_codigo + i, b)

df = df[cols]


# ============================
#  EXPORTAR
# ============================

ruta_salida = r"C:\Users\DeVan\Documents\Jupyterlab\ventas\ventas_preparadas.csv"
df.to_csv(ruta_salida, sep=";", index=False, encoding="utf-8-sig")

print("\033[92m=========  Ejecutado correctamente  =========\033[0m")





